### 本节课调用adaboost以及基于简单决策树实现adaboost模型  

#### 我们将利用adaboost模型对肿瘤类型进行判断与分类

- 数据来源： https://www.kaggle.com/uciml/breast-cancer-wisconsin-data  
- 对比模型： Adaboost模型

In [ ]:
# necessary imports
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

np.random.seed(19)

In [ ]:
data_folder = "../input"
data = pd.read_csv(os.path.join(data_folder, "breastCancer.csv"))

In [ ]:
data.head()

#### 检查需要预测的目标

In [ ]:
data['diagnosis'].value_counts()

#### 过滤不需要的信息

In [ ]:
data.drop('id',axis=1,inplace=True)
data.drop('Unnamed: 32',axis=1,inplace=True)

#### 转换预测的目标   M：1， B：-1

In [ ]:
data['diagnosis'] = data['diagnosis'].apply(lambda x : +1 if x=='M' else -1)

#### 观察数据的基本信息

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
import seaborn as sns
sns.countplot(data['diagnosis'])

#### 利用前6个特征，设定目标变量

In [ ]:
features = data.columns[1:7]
target = 'diagnosis'
features

#### 特征探索  

- 比较6个特征与恶性肿瘤的关系

In [ ]:
i = 0
for feature in features:

    bins = 25
    # 将特征的直方图画出来
    plt.hist(data[feature], bins=bins, color='lightblue', label= 'Total', alpha=1)
    plt.hist(data[feature][data[target] == 1], bins=bins, color='k', label='M-bad', alpha=0.5)
    
    plt.xlabel(feature)
    plt.ylabel('Amount of count')
    
    plt.legend()
    
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.3)

In [ ]:
trainX, trainY = train_data[data.columns[1:]], train_data[target]
testX, testY = test_data[data.columns[1:]], test_data[target]

### Logistic回归的表现

In [ ]:
logistic_model = LogisticRegression()
print("Logistic Regression performance: %f" % (cross_val_score(logistic_model, trainX, trainY, cv=8).mean()))

### 决策树的表现

In [ ]:
tree_model = DecisionTreeClassifier()
print("Decision Tree performance: %f" % (cross_val_score(tree_model, trainX, trainY, cv=8).mean()))

### 直接调用adaboost模型的表现

In [ ]:
ada_model = AdaBoostClassifier(n_estimators=200)
print("Decision Tree performance: %f" % (cross_val_score(ada_model, trainX, trainY, cv=8).mean()))

### 测试集的表现

In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(trainX, trainY)
print("Logistic Regression test performance: %f" % logistic_model.score(testX, testY))

In [ ]:
tree_model = DecisionTreeClassifier()
tree_model.fit(trainX, trainY)
print("Decision Tree test performance: %f" % tree_model.score(testX, testY))

In [ ]:
ada_model = AdaBoostClassifier(n_estimators=200)
ada_model.fit(trainX, trainY)
print("Adaboost test performance: %f" % ada_model.score(testX, testY))

#### Adaboost的实现

In [ ]:
from sklearn.base import BaseEstimator
class Adaboost(BaseEstimator):
    
    def __init__(self, M):
        self.M = M
        
    def fit(self, X, Y):
        self.models = []
        self.model_weights = []
        
        N, _ = X.shape
        alpha = np.ones(N) / N
        
        for m in range(self.M):
            tree = DecisionTreeClassifier(max_depth=2)
            tree.fit(X, Y, sample_weight=alpha)
            prediction = tree.predict(X)
            
            # 计算加权错误
            weighted_error = alpha.dot(prediction != Y)
            
            # 计算当前模型的权重
            model_weight = 0.5 * (np.log(1 - weighted_error) - np.log(weighted_error))
            
            # 更新数据的权重
            alpha = alpha * np.exp(-model_weight * Y * prediction)
            
            # 数据权重normalize
            alpha = alpha / alpha.sum()
            
            self.models.append(tree)
            self.model_weights.append(model_weight)
            
    def predict(self, X):
        N, _ = X.shape
        result = np.zeros(N)
        for wt, tree in zip(self.model_weights, self.models):
            result += wt * tree.predict(X)
        
        return np.sign(result)
    
    def score(self, X, Y):
        prediction = self.predict(X)
        return np.mean(prediction == Y)

### Adaboost的表现

In [ ]:
adamodel = Adaboost(200)
print("Adaboost model performance: %f" % (cross_val_score(adamodel, trainX.as_matrix().astype(np.float64), trainY.as_matrix().astype(np.float64), cv=8).mean()))

### 测试集的表现

In [ ]:
adamodel.fit(trainX.as_matrix().astype(np.float64), trainY.as_matrix().astype(np.float64))
print("Adaboost model test performance: %f" % adamodel.score(testX.as_matrix().astype(np.float64), testY.as_matrix().astype(np.float64)))